# 开始实战

In [1]:
%store -r train_housing_labels
%store -r train_housing_prepared

/home/jimmyli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jimmyli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### 7、选择及训练模型

#### 线性回归模型(LinearRegression)

In [2]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(train_housing_prepared, train_housing_labels)

/home/jimmyli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

训练完成，然后评估模型，计算训练集中的均方根误差(RMSE)

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(train_housing_prepared)
lin_mse = mean_squared_error(train_housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848922

#### 决策树模型(DecisionTreeRegressor)

In [4]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_housing_prepared, train_housing_labels)
housing_predictions = tree_reg.predict(train_housing_prepared)
tree_mse = mean_squared_error(train_housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

/home/jimmyli/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0.0

可以看到决策树回归模型的的训练集均方误差竟然为0。比线性回归模型的的训练集均方误差小太多太多。  
但这是否说明了决策树回归模型比线性回归模型在此问题上好很多，当然不是，训练误差小的模型并不代表为好模型，这是因为模型可能过度地学习了训练集的数据，只是在训练集上的表现好（即过拟合），一旦测试新的数据表现就会很差。  
因此在训练的时候需要将部分的训练数据提取出来作为验证集，验证该模型是否对此问题适用。其中比较常用的就是交叉验证法。 

#### 交叉验证法

在Scikit-Learn中交叉验证对应的类为cross_val_score，下面是线性回归模型与决策树回归模型的交叉验证实例：

In [5]:
from sklearn.model_selection import cross_val_score
tree_scores = cross_val_score(tree_reg, train_housing_prepared, train_housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_scores = cross_val_score(lin_reg, train_housing_prepared, train_housing_labels,
                            scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
lin_rmse_scores = np.sqrt(-lin_scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)
display_scores(lin_rmse_scores)

Scores: [69455.04581656 67684.3066084  69403.81676552 70136.42707744
 72166.49955535 76178.96591191 70679.4667266  71980.60722948
 77487.58893325 69872.86923609]
Mean: 71504.55938606
Standard deviation: 2945.0711135561087
Scores: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean: 69052.46136345083
Standard deviation: 2731.674001798348


可以看到决策树回归模型的交叉验证平均误差为71163，而线性回归模型的交叉验证平均误差为69051，这说明决策树回归模型明显是过拟合，实际上比线性回归模型要差一些。  
除了这两个简单的模型以外，还应该试验不同的模型（如随机森林，不同核的SVM，神经网络等），最终选择2-5个候选的模型。（也可以写到同一个文件下，方便以后直接调用）

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### 保存模型

In [6]:
from sklearn.externals import joblib
joblib.dump(lin_reg, "lin-reg.pkl")
# and later...
#my_model_loaded = joblib.load("my_model.pkl")  加载模型

['lin-reg.pkl']

## 8、模型调参

#### 网格搜索 Gird Search

scikit-learn中提供函数GridSearchCV用于网格搜索调参，网格搜索就是通过自己对模型需要调整的几个参数设定一些可行值，然后Grid Search会排列组合这些参数值，每一种情况都去训练一个模型，经过交叉验证后输出结果。下面为随机森林回归模型（RandomForestRegression）的一个Grid Search的例子。

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = Rando